In [ ]:
import sys
import ee
%run Sentinel2b.ipynb
#ee.Authenticate()
ee.Initialize()

modulename = 'ee'
if modulename not in sys.modules:
    # import GEE and Authenticate, token or log in will be asked from web browser
    import ee
    # TO DO : UNCOMMENT THIS IF IT IS THE FIRST TIME AUTHINTICATE GEE ON THIS MACHINE
    # ee.Authenticate()
    ee.Initialize()
# else:
    # google earth engine already imported and authenticated


modulename = 'ipynb_MapVisualisation'
if modulename not in sys.modules:
    %run MapVisualisation.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_MapVisualisation'] = None
# else
    # Utils modules has already been loaded somewhere else


modulename = 'ipynb_Sentinel2b'
if modulename not in sys.modules:
    %run Sentinel2b.ipynb
    # adding an identifier to sys.modules to avoiding loading the same file multiple times
    sys.modules['ipynb_Sentinel2b'] = None
# else
    # Utils modules has already been loaded somewhere else


In [ ]:
%run MapVisualisation.ipynb
# These are the coordinates of Spain
my_map = folium.Map(location=[42, -3], zoom_start=11)
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


def update_cld_shdw_mask(img):
    print('test')


countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
AOI = countries.filter(ee.Filter.eq('country_na', 'Spain'))

AOI = ee.Geometry.Polygon(
        [[[-3.2242859153727754, 40.999456684741496],
          [-3.2242859153727754, 40.928422009540796],
          [-3.121975734708713, 40.928422009540796],
          [-3.121975734708713, 40.999456684741496]]])


AOI = ee.Geometry.Polygon(
        [[[-2.97404756802236, 41.99472813320794],
          [-2.97404756802236, 41.8107480842418],
          [-2.66643038052236, 41.8107480842418],
          [-2.66643038052236, 41.99472813320794]]])

# cloudmask01 = getCloudMask(median)

START_DATE = '2019-01-01'
END_DATE = '2019-12-31'



masks = { 'gsw': 1}#,'forestMask': {'buffer':10, 'startDate':'2000-01-01', 'endDate':'2022-12-31',}} # # , 'lmask': 30,


s2 = Sentinel2(AOI, START_DATE, END_DATE, 50, masks)
# s2.byMonth(2020)

medianS2 = s2.getMedian()
bandNames = medianS2.bandNames()
print('Band names medianS2: ', bandNames.getInfo())
med = s2.getCollection().first()

# Subset layers and prepare them for display.
clouds = med.select('clouds').selfMask()
shadows = med.select('shadows').selfMask()


dark_pixels = med.select('dark_pixels')  # .selfMask()
probability = med.select('probability')
cloudmask = med.select('cloudmask')
cloudmaskMasked = cloudmask  # .selfMask()
cloud_transform = med.select('cloud_transform')

medianS2 = medianS2.select(['B4', 'B3', 'B2'])

task = ee.batch.Export.image.toDrive(**{
    'image': medianS2,
    'description': 'medRGB1_GB1_2019',
    'folder': 'earth_engine_demos',
    'scale': 10,
    'region': AOI.getInfo()['coordinates'],
    'maxPixels': 1549491660
})
task.start()


# Add Land Mask to the map
my_map.add_ee_layer(medianS2, {
                    'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'median')
my_map.add_ee_layer(AOI, {}, 'AOI')



# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)
